<a href="https://colab.research.google.com/github/Nandhu-666/banking-system-using-python/blob/main/AI_Personal_Finance_Advisor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import gradio as gr
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

class PersonalFinanceAdvisor:
    def __init__(self):
        """
        Initialize the Personal Finance Advisor with empty data structures
        and machine learning models for expense prediction
        """
        self.expenses_df = pd.DataFrame()
        self.model = RandomForestRegressor(n_estimators=100, random_state=42)
        self.label_encoder = LabelEncoder()
        self.is_model_trained = False
        self.budget_limits = {}

        # Predefined expense categories for classification
        self.expense_categories = {
            'food': ['restaurant', 'grocery', 'food', 'eat', 'pizza', 'coffee', 'lunch', 'dinner'],
            'transportation': ['uber', 'taxi', 'gas', 'fuel', 'parking', 'bus', 'metro', 'train'],
            'entertainment': ['movie', 'game', 'concert', 'netflix', 'spotify', 'entertainment'],
            'shopping': ['amazon', 'shop', 'clothes', 'mall', 'store', 'purchase'],
            'utilities': ['electricity', 'water', 'gas bill', 'phone', 'internet', 'utility'],
            'healthcare': ['doctor', 'medicine', 'hospital', 'pharmacy', 'health'],
            'education': ['course', 'book', 'tuition', 'school', 'university', 'education'],
            'other': []
        }

    def categorize_expense(self, description):
        """
        Automatically categorize expenses based on description using keyword matching

        Args:
            description (str): Description of the expense

        Returns:
            str: Category of the expense
        """
        description_lower = description.lower()

        for category, keywords in self.expense_categories.items():
            if any(keyword in description_lower for keyword in keywords):
                return category

        return 'other'

    def add_expense(self, amount, description, date=None):
        """
        Add a new expense to the system with automatic categorization

        Args:
            amount (float): Amount spent
            description (str): Description of the expense
            date (str, optional): Date of expense. If None, uses current date

        Returns:
            str: Confirmation message
        """
        if date is None:
            date = datetime.now().strftime('%Y-%m-%d')

        # Automatically categorize the expense
        category = self.categorize_expense(description)

        # Create new expense entry
        new_expense = {
            'date': date,
            'amount': float(amount),
            'description': description,
            'category': category,
            'day_of_week': pd.to_datetime(date).dayofweek,
            'month': pd.to_datetime(date).month
        }

        # Add to DataFrame
        self.expenses_df = pd.concat([self.expenses_df, pd.DataFrame([new_expense])], ignore_index=True)

        # Retrain model if we have enough data
        if len(self.expenses_df) >= 10:
            self.train_prediction_model()

        return f"✅ Added expense: ${amount} for {description} (Category: {category})"

    def train_prediction_model(self):
        """
        Train a machine learning model to predict future expenses based on historical data
        Uses Random Forest Regressor with features like category, day of week, and month
        """
        if len(self.expenses_df) < 10:
            return "Need at least 10 expenses to train the model"

        # Prepare features for machine learning
        df_model = self.expenses_df.copy()

        # Encode categorical variables
        df_model['category_encoded'] = self.label_encoder.fit_transform(df_model['category'])

        # Features: category, day of week, month
        X = df_model[['category_encoded', 'day_of_week', 'month']]
        y = df_model['amount']

        # Train the model
        self.model.fit(X, y)
        self.is_model_trained = True

        return "Model trained successfully!"

    def predict_next_expense(self, category, days_ahead=7):
        """
        Predict the next expense amount for a given category

        Args:
            category (str): Expense category
            days_ahead (int): Number of days ahead to predict for

        Returns:
            float: Predicted expense amount
        """
        if not self.is_model_trained:
            return 0.0

        future_date = datetime.now() + timedelta(days=days_ahead)

        # Encode the category
        if category in self.label_encoder.classes_:
            category_encoded = self.label_encoder.transform([category])[0]
        else:
            category_encoded = 0

        # Prepare features
        features = np.array([[category_encoded, future_date.weekday(), future_date.month]])

        # Make prediction
        prediction = self.model.predict(features)[0]
        return max(0, prediction)  # Ensure non-negative prediction

    def generate_spending_analysis(self):
        """
        Generate comprehensive spending analysis with visualizations

        Returns:
            tuple: (analysis_text, matplotlib_figure)
        """
        if self.expenses_df.empty:
            return "No expenses recorded yet. Add some expenses to see analysis!", None

        # Calculate statistics
        total_spent = self.expenses_df['amount'].sum()
        avg_expense = self.expenses_df['amount'].mean()
        most_expensive = self.expenses_df.loc[self.expenses_df['amount'].idxmax()]

        # Category-wise spending
        category_spending = self.expenses_df.groupby('category')['amount'].sum().sort_values(ascending=False)

        # Create visualizations
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

        # 1. Category-wise spending pie chart
        ax1.pie(category_spending.values, labels=category_spending.index, autopct='%1.1f%%', startangle=90)
        ax1.set_title('Spending by Category')

        # 2. Daily spending trend
        daily_spending = self.expenses_df.groupby('date')['amount'].sum()
        ax2.plot(daily_spending.index, daily_spending.values, marker='o')
        ax2.set_title('Daily Spending Trend')
        ax2.set_xlabel('Date')
        ax2.set_ylabel('Amount ($)')
        ax2.tick_params(axis='x', rotation=45)

        # 3. Category-wise bar chart
        ax3.bar(category_spending.index, category_spending.values)
        ax3.set_title('Total Spending by Category')
        ax3.set_xlabel('Category')
        ax3.set_ylabel('Amount ($)')
        ax3.tick_params(axis='x', rotation=45)

        # 4. Monthly spending pattern
        monthly_spending = self.expenses_df.groupby('month')['amount'].sum()
        months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
        month_labels = [months[i-1] for i in monthly_spending.index]
        ax4.bar(month_labels, monthly_spending.values, color='skyblue')
        ax4.set_title('Monthly Spending Pattern')
        ax4.set_xlabel('Month')
        ax4.set_ylabel('Amount ($)')

        plt.tight_layout()

        # Generate analysis text
        analysis_text = f"""
        📊 **SPENDING ANALYSIS REPORT**

        💰 **Overview:**
        • Total Spent: ${total_spent:.2f}
        • Average Expense: ${avg_expense:.2f}
        • Number of Transactions: {len(self.expenses_df)}

        🏆 **Highest Expense:**
        • Amount: ${most_expensive['amount']:.2f}
        • Description: {most_expensive['description']}
        • Category: {most_expensive['category']}

        📈 **Top Spending Categories:**
        """

        for category, amount in category_spending.head(3).items():
            percentage = (amount / total_spent) * 100
            analysis_text += f"\n• {category.capitalize()}: ${amount:.2f} ({percentage:.1f}%)"

        return analysis_text, fig

    def predict_future_spending(self, days_ahead=30):
        """
        Predict future spending patterns and provide budget recommendations

        Args:
            days_ahead (int): Number of days to predict ahead

        Returns:
            str: Prediction report with recommendations
        """
        if not self.is_model_trained:
            return "❌ Need more data to make predictions. Add at least 10 expenses!"

        predictions = {}
        total_predicted = 0

        # Get unique categories from historical data
        categories = self.expenses_df['category'].unique()

        for category in categories:
            # Predict spending for each category
            predicted_amount = self.predict_next_expense(category, days_ahead)
            predictions[category] = predicted_amount
            total_predicted += predicted_amount

        # Calculate current spending rate
        if len(self.expenses_df) > 0:
            current_daily_avg = self.expenses_df['amount'].sum() / len(self.expenses_df['date'].unique())
        else:
            current_daily_avg = 0

        # Generate recommendations
        report = f"""
        🔮 **FUTURE SPENDING PREDICTIONS** (Next {days_ahead} days)

        💡 **Predicted Total Spending:** ${total_predicted:.2f}
        📊 **Current Daily Average:** ${current_daily_avg:.2f}

        📋 **Category-wise Predictions:**
        """

        for category, amount in sorted(predictions.items(), key=lambda x: x[1], reverse=True):
            if amount > 0:
                report += f"\n• {category.capitalize()}: ${amount:.2f}"

        # Budget recommendations
        report += f"""

        💰 **BUDGET RECOMMENDATIONS:**
        • Set aside ${total_predicted * 1.1:.2f} for the next {days_ahead} days (10% buffer)
        • Consider reducing spending in top categories if needed
        • Track daily spending to stay within ${total_predicted/days_ahead:.2f} per day

        ⚠️ **ALERTS:**
        """

        # Alert for high spending categories
        avg_category_spending = total_predicted / len(categories)
        for category, amount in predictions.items():
            if amount > avg_category_spending * 1.5:
                report += f"\n• High predicted spending in {category}: ${amount:.2f}"

        return report

    def detect_spending_anomalies(self):
        """
        Detect unusual spending patterns that might indicate overspending

        Returns:
            str: Anomaly detection report
        """
        if len(self.expenses_df) < 5:
            return "Need more data to detect anomalies"

        # Calculate spending statistics
        mean_amount = self.expenses_df['amount'].mean()
        std_amount = self.expenses_df['amount'].std()
        threshold = mean_amount + (2 * std_amount)  # 2 standard deviations above mean

        # Find anomalies
        anomalies = self.expenses_df[self.expenses_df['amount'] > threshold]

        if anomalies.empty:
            return "✅ No unusual spending detected. Your spending patterns look normal!"

        report = f"""
        🚨 **SPENDING ANOMALY ALERT**

        📊 **Statistics:**
        • Average Expense: ${mean_amount:.2f}
        • Spending Threshold: ${threshold:.2f}
        • Anomalies Found: {len(anomalies)}

        ⚠️ **Unusual Expenses:**
        """

        for _, expense in anomalies.iterrows():
            report += f"\n• ${expense['amount']:.2f} - {expense['description']} ({expense['date']})"

        report += f"""

        💡 **Recommendations:**
        • Review these high expenses to ensure they were necessary
        • Consider setting spending limits for categories with anomalies
        • Monitor future spending in these categories more closely
        """

        return report

# Initialize the finance advisor
finance_advisor = PersonalFinanceAdvisor()

# Gradio Interface Functions
def add_expense_interface(amount, description, date):
    """Interface function for adding expenses through Gradio"""
    try:
        result = finance_advisor.add_expense(amount, description, date)
        return result
    except Exception as e:
        return f"Error: {str(e)}"

def get_analysis_interface():
    """Interface function for getting spending analysis through Gradio"""
    try:
        analysis_text, fig = finance_advisor.generate_spending_analysis()
        return analysis_text, fig
    except Exception as e:
        return f"Error: {str(e)}", None

def get_predictions_interface(days):
    """Interface function for getting spending predictions through Gradio"""
    try:
        predictions = finance_advisor.predict_future_spending(days)
        return predictions
    except Exception as e:
        return f"Error: {str(e)}"

def get_anomalies_interface():
    """Interface function for detecting spending anomalies through Gradio"""
    try:
        anomalies = finance_advisor.detect_spending_anomalies()
        return anomalies
    except Exception as e:
        return f"Error: {str(e)}"

# Create Gradio Interface
def create_gradio_interface():
    """
    Create and configure the Gradio interface for the Personal Finance Advisor
    """
    with gr.Blocks(title="AI Personal Finance Advisor", theme=gr.themes.Soft()) as app:
        gr.Markdown("""
        # 🤖 AI Personal Finance Advisor

        Welcome to your intelligent financial companion! This system helps you:
        - 📝 Track expenses with automatic categorization
        - 📊 Analyze spending patterns with visual insights
        - 🔮 Predict future expenses using machine learning
        - 🚨 Detect unusual spending anomalies
        """)

        with gr.Tab("💰 Add Expense"):
            gr.Markdown("### Add a new expense to your financial records")

            with gr.Row():
                amount_input = gr.Number(label="Amount ($)", value=0.0)
                description_input = gr.Textbox(label="Description", placeholder="e.g., Lunch at restaurant")
                date_input = gr.Textbox(label="Date (YYYY-MM-DD)", value=datetime.now().strftime('%Y-%m-%d'))

            add_button = gr.Button("Add Expense", variant="primary")
            add_output = gr.Textbox(label="Result", lines=2)

            add_button.click(
                add_expense_interface,
                inputs=[amount_input, description_input, date_input],
                outputs=add_output
            )

        with gr.Tab("📊 Spending Analysis"):
            gr.Markdown("### Comprehensive analysis of your spending patterns")

            analysis_button = gr.Button("Generate Analysis", variant="primary")
            analysis_output = gr.Textbox(label="Analysis Report", lines=15)
            analysis_plot = gr.Plot(label="Spending Visualizations")

            analysis_button.click(
                get_analysis_interface,
                outputs=[analysis_output, analysis_plot]
            )

        with gr.Tab("🔮 Future Predictions"):
            gr.Markdown("### AI-powered predictions for your future spending")

            days_input = gr.Slider(minimum=7, maximum=90, value=30, label="Predict for next X days")
            predict_button = gr.Button("Generate Predictions", variant="primary")
            predictions_output = gr.Textbox(label="Predictions & Recommendations", lines=15)

            predict_button.click(
                get_predictions_interface,
                inputs=[days_input],
                outputs=predictions_output
            )

        with gr.Tab("🚨 Anomaly Detection"):
            gr.Markdown("### Detect unusual spending patterns and potential overspending")

            anomaly_button = gr.Button("Detect Anomalies", variant="primary")
            anomaly_output = gr.Textbox(label="Anomaly Report", lines=10)

            anomaly_button.click(
                get_anomalies_interface,
                outputs=anomaly_output
            )

        gr.Markdown("""
        ---
        ### 📝 How to Use:
        1. **Add Expenses**: Start by adding your daily expenses with descriptions
        2. **View Analysis**: Get insights into your spending patterns with charts
        3. **Future Predictions**: Let AI predict your future spending and get budget recommendations
        4. **Anomaly Detection**: Identify unusual spending that might need attention

        ### 🎯 Features:
        - **Automatic Categorization**: Expenses are automatically categorized using smart keyword matching
        - **Machine Learning**: Random Forest model predicts future spending based on your patterns
        - **Visual Analytics**: Beautiful charts and graphs show your spending trends
        - **Smart Alerts**: Get notified about unusual spending patterns
        """)

    return app

# Launch the application
if __name__ == "__main__":
    app = create_gradio_interface()
    app.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://b7ccb3d9fb978a2749.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://b7ccb3d9fb978a2749.gradio.live
